In [1]:
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import SQLContext


#from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry


spark = SparkSession.builder.appName("hw1_011504272(method_optimized)").getOrCreate()
data_rdd = spark.read.text("C:\\Users\\kpgaj\\train.dat").rdd
splitted = data_rdd.map(lambda row: row.value.split("\t"))
splitted.collect()
rrdd = splitted.map(lambda a: Row(UserID=int(a[0]),MovieID=int(a[1]), Rating=float(a[2]), Timestamp=int(a[3])))
data_frame_spark = spark.createDataFrame(rrdd).cache()

In [2]:
import pandas as pd
pandas_df = data_frame_spark.toPandas()

In [3]:
pandas_df.describe()

,MovieID,Rating,Timestamp,UserID
count,85724.000000,85724.000000,8.572400e+04,85724.000000
mean,1274.367318,3.530353,8.839002e+08,554.367108
std,330.674056,1.124838,5.330935e+06,256.646885
min,20.000000,1.000000,8.750929e+08,57.000000
25%,1069.000000,3.000000,8.798194e+08,347.000000
50%,1378.000000,4.000000,8.832115e+08,574.000000
75%,1525.000000,4.000000,8.885736e+08,751.000000
max,1699.000000,5.000000,8.936548e+08,999.000000


In [4]:
import surprise
df_train= pandas_df.drop(columns = 'Timestamp')

from surprise import Reader
reader = Reader()
d_data =surprise.Dataset.load_from_df(df_train[['MovieID', 'UserID', 'Rating']], reader) 

data = d_data.build_full_trainset()



In [5]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(d_data, test_size=.15)

In [6]:

from surprise import accuracy
from surprise.model_selection import GridSearchCV
from surprise import SVD
from surprise.model_selection import cross_validate
param_grid = {'n_factors': [120,200,250,180], 'n_epochs': [100,500,140], 'lr_all': [0.023, 0.004,0.054],'reg_all': [0.01,0.2,0.05]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)


In [7]:
gs.fit(d_data)
params = gs.best_params['rmse']
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

svdtuned = SVD(n_factors=params['n_factors'], n_epochs=params['n_epochs'],lr_all=params['lr_all'], reg_all=params['reg_all'])


0.9328598676062377
{'n_factors': 200, 'n_epochs': 500, 'lr_all': 0.004, 'reg_all': 0.2}


In [9]:
svdtuned.fit(trainset)

test_pred = svdtuned.test(testset)
#print("SVD : Test Set")
accuracy.rmse(test_pred, verbose=True)


RMSE: 0.9375


0.9374834120000148

In [10]:
list_actual =[]
list_pred =[]
for i in range(len(testset)) :
    i = pandas_df.loc[i,"UserID"]
    j = pandas_df.loc[i,"MovieID"]
    k = pandas_df.loc[i,"Rating"]
    list_actual.append(k)
    list_pred.append(svdtuned.predict(i,j,k).est)
    print(i,j,k, svdtuned.predict(i,j,k)) 
    

905 1422 4.0 user: 905        item: 1422       r_ui = 4.00   est = 2.67   {'was_impossible': False}
697 1465 5.0 user: 697        item: 1465       r_ui = 5.00   est = 2.68   {'was_impossible': False}
855 1671 3.0 user: 855        item: 1671       r_ui = 3.00   est = 3.45   {'was_impossible': False}
950 1051 1.0 user: 950        item: 1051       r_ui = 1.00   est = 3.75   {'was_impossible': False}
806 1207 4.0 user: 806        item: 1207       r_ui = 4.00   est = 2.41   {'was_impossible': False}
855 1671 3.0 user: 855        item: 1671       r_ui = 3.00   est = 3.45   {'was_impossible': False}
778 1154 3.0 user: 778        item: 1154       r_ui = 3.00   est = 3.72   {'was_impossible': False}
993 1157 2.0 user: 993        item: 1157       r_ui = 2.00   est = 3.85   {'was_impossible': False}
891 1297 1.0 user: 891        item: 1297       r_ui = 1.00   est = 3.12   {'was_impossible': False}
910 1417 5.0 user: 910        item: 1417       r_ui = 5.00   est = 3.06   {'was_impossible': False}


870 1289 2.0 user: 870        item: 1289       r_ui = 2.00   est = 2.21   {'was_impossible': False}
812 1141 3.0 user: 812        item: 1141       r_ui = 3.00   est = 3.27   {'was_impossible': False}
944 1688 5.0 user: 944        item: 1688       r_ui = 5.00   est = 2.90   {'was_impossible': False}
944 1688 5.0 user: 944        item: 1688       r_ui = 5.00   est = 2.90   {'was_impossible': False}
980 1392 5.0 user: 980        item: 1392       r_ui = 5.00   est = 2.99   {'was_impossible': False}
980 1392 5.0 user: 980        item: 1392       r_ui = 5.00   est = 2.99   {'was_impossible': False}
778 1154 3.0 user: 778        item: 1154       r_ui = 3.00   est = 3.72   {'was_impossible': False}
951 1181 5.0 user: 951        item: 1181       r_ui = 5.00   est = 3.16   {'was_impossible': False}
693 1430 4.0 user: 693        item: 1430       r_ui = 4.00   est = 4.01   {'was_impossible': False}
856 1172 4.0 user: 856        item: 1172       r_ui = 4.00   est = 3.38   {'was_impossible': False}


673 1567 4.0 user: 673        item: 1567       r_ui = 4.00   est = 3.17   {'was_impossible': False}
709 1214 3.0 user: 709        item: 1214       r_ui = 3.00   est = 2.79   {'was_impossible': False}
709 1214 3.0 user: 709        item: 1214       r_ui = 3.00   est = 2.79   {'was_impossible': False}
754 1652 5.0 user: 754        item: 1652       r_ui = 5.00   est = 3.23   {'was_impossible': False}
989 1058 4.0 user: 989        item: 1058       r_ui = 4.00   est = 3.59   {'was_impossible': False}
671 1298 2.0 user: 671        item: 1298       r_ui = 2.00   est = 2.23   {'was_impossible': False}
914 1161 4.0 user: 914        item: 1161       r_ui = 4.00   est = 3.09   {'was_impossible': False}
993 1157 2.0 user: 993        item: 1157       r_ui = 2.00   est = 3.85   {'was_impossible': False}
754 1652 5.0 user: 754        item: 1652       r_ui = 5.00   est = 3.23   {'was_impossible': False}
692 1445 1.0 user: 692        item: 1445       r_ui = 1.00   est = 3.53   {'was_impossible': False}


748 1084 2.0 user: 748        item: 1084       r_ui = 2.00   est = 3.13   {'was_impossible': False}
779 946 4.0 user: 779        item: 946        r_ui = 4.00   est = 4.17   {'was_impossible': False}
708 1586 5.0 user: 708        item: 1586       r_ui = 5.00   est = 2.65   {'was_impossible': False}
823 953 5.0 user: 823        item: 953        r_ui = 5.00   est = 2.45   {'was_impossible': False}
958 1291 2.0 user: 958        item: 1291       r_ui = 2.00   est = 3.57   {'was_impossible': False}
999 1525 4.0 user: 999        item: 1525       r_ui = 4.00   est = 3.82   {'was_impossible': False}
987 1376 3.0 user: 987        item: 1376       r_ui = 3.00   est = 3.59   {'was_impossible': False}
924 1623 3.0 user: 924        item: 1623       r_ui = 3.00   est = 3.57   {'was_impossible': False}
731 1315 4.0 user: 731        item: 1315       r_ui = 4.00   est = 3.68   {'was_impossible': False}
997 1331 2.0 user: 997        item: 1331       r_ui = 2.00   est = 3.46   {'was_impossible': False}
71

891 1297 1.0 user: 891        item: 1297       r_ui = 1.00   est = 3.12   {'was_impossible': False}
832 1516 4.0 user: 832        item: 1516       r_ui = 4.00   est = 3.42   {'was_impossible': False}
799 1573 5.0 user: 799        item: 1573       r_ui = 5.00   est = 2.40   {'was_impossible': False}
778 1154 3.0 user: 778        item: 1154       r_ui = 3.00   est = 3.72   {'was_impossible': False}
885 1402 4.0 user: 885        item: 1402       r_ui = 4.00   est = 3.23   {'was_impossible': False}
666 1127 1.0 user: 666        item: 1127       r_ui = 1.00   est = 2.60   {'was_impossible': False}
881 1621 4.0 user: 881        item: 1621       r_ui = 4.00   est = 2.75   {'was_impossible': False}
802 1699 4.0 user: 802        item: 1699       r_ui = 4.00   est = 3.20   {'was_impossible': False}
909 1382 5.0 user: 909        item: 1382       r_ui = 5.00   est = 2.76   {'was_impossible': False}
938 1400 2.0 user: 938        item: 1400       r_ui = 2.00   est = 3.35   {'was_impossible': False}


...

591 813 4.0 user: 591        item: 813        r_ui = 4.00   est = 3.64   {'was_impossible': False}
806 1207 4.0 user: 806        item: 1207       r_ui = 4.00   est = 2.41   {'was_impossible': False}
799 1573 5.0 user: 799        item: 1573       r_ui = 5.00   est = 2.40   {'was_impossible': False}
826 1442 5.0 user: 826        item: 1442       r_ui = 5.00   est = 3.23   {'was_impossible': False}
744 707 3.0 user: 744        item: 707        r_ui = 3.00   est = 3.32   {'was_impossible': False}
936 1543 3.0 user: 936        item: 1543       r_ui = 3.00   est = 2.89   {'was_impossible': False}
622 1001 4.0 user: 622        item: 1001       r_ui = 4.00   est = 2.99   {'was_impossible': False}
944 1688 5.0 user: 944        item: 1688       r_ui = 5.00   est = 2.90   {'was_impossible': False}
944 1688 5.0 user: 944        item: 1688       r_ui = 5.00   est = 2.90   {'was_impossible': False}
766 1134 4.0 user: 766        item: 1134       r_ui = 4.00   est = 2.93   {'was_impossible': False}
59

591 813 4.0 user: 591        item: 813        r_ui = 4.00   est = 3.64   {'was_impossible': False}
999 1525 4.0 user: 999        item: 1525       r_ui = 4.00   est = 3.82   {'was_impossible': False}
772 553 4.0 user: 772        item: 553        r_ui = 4.00   est = 3.10   {'was_impossible': False}
966 1579 2.0 user: 966        item: 1579       r_ui = 2.00   est = 2.65   {'was_impossible': False}
885 1402 4.0 user: 885        item: 1402       r_ui = 4.00   est = 3.23   {'was_impossible': False}
947 1346 3.0 user: 947        item: 1346       r_ui = 3.00   est = 3.92   {'was_impossible': False}
587 954 4.0 user: 587        item: 954        r_ui = 4.00   est = 3.76   {'was_impossible': False}
848 1296 3.0 user: 848        item: 1296       r_ui = 3.00   est = 2.23   {'was_impossible': False}
607 540 5.0 user: 607        item: 540        r_ui = 5.00   est = 3.33   {'was_impossible': False}
670 1290 3.0 user: 670        item: 1290       r_ui = 3.00   est = 2.37   {'was_impossible': False}
981 

792 1103 4.0 user: 792        item: 1103       r_ui = 4.00   est = 2.81   {'was_impossible': False}
800 1443 4.0 user: 800        item: 1443       r_ui = 4.00   est = 3.86   {'was_impossible': False}
594 898 3.0 user: 594        item: 898        r_ui = 3.00   est = 2.32   {'was_impossible': False}
618 1552 1.0 user: 618        item: 1552       r_ui = 1.00   est = 3.01   {'was_impossible': False}
601 1407 5.0 user: 601        item: 1407       r_ui = 5.00   est = 3.68   {'was_impossible': False}
731 1315 4.0 user: 731        item: 1315       r_ui = 4.00   est = 3.68   {'was_impossible': False}
595 1634 4.0 user: 595        item: 1634       r_ui = 4.00   est = 3.02   {'was_impossible': False}
802 1699 4.0 user: 802        item: 1699       r_ui = 4.00   est = 3.20   {'was_impossible': False}
693 1430 4.0 user: 693        item: 1430       r_ui = 4.00   est = 4.01   {'was_impossible': False}
870 1289 2.0 user: 870        item: 1289       r_ui = 2.00   est = 2.21   {'was_impossible': False}
6

787 1541 3.0 user: 787        item: 1541       r_ui = 3.00   est = 2.20   {'was_impossible': False}
823 953 5.0 user: 823        item: 953        r_ui = 5.00   est = 2.45   {'was_impossible': False}
736 1442 5.0 user: 736        item: 1442       r_ui = 5.00   est = 3.58   {'was_impossible': False}
607 540 5.0 user: 607        item: 540        r_ui = 5.00   est = 3.33   {'was_impossible': False}
951 1181 5.0 user: 951        item: 1181       r_ui = 5.00   est = 3.16   {'was_impossible': False}
898 1015 3.0 user: 898        item: 1015       r_ui = 3.00   est = 2.98   {'was_impossible': False}
917 1672 3.0 user: 917        item: 1672       r_ui = 3.00   est = 2.81   {'was_impossible': False}
637 683 5.0 user: 637        item: 683        r_ui = 5.00   est = 3.94   {'was_impossible': False}
745 1515 4.0 user: 745        item: 1515       r_ui = 4.00   est = 3.43   {'was_impossible': False}
911 1233 4.0 user: 911        item: 1233       r_ui = 4.00   est = 3.44   {'was_impossible': False}
603

868 1432 3.0 user: 868        item: 1432       r_ui = 3.00   est = 2.59   {'was_impossible': False}
906 1429 3.0 user: 906        item: 1429       r_ui = 3.00   est = 3.40   {'was_impossible': False}
917 1672 3.0 user: 917        item: 1672       r_ui = 3.00   est = 2.81   {'was_impossible': False}
707 1631 5.0 user: 707        item: 1631       r_ui = 5.00   est = 3.44   {'was_impossible': False}
637 683 5.0 user: 637        item: 683        r_ui = 5.00   est = 3.94   {'was_impossible': False}
770 517 2.0 user: 770        item: 517        r_ui = 2.00   est = 2.25   {'was_impossible': False}
944 1688 5.0 user: 944        item: 1688       r_ui = 5.00   est = 2.90   {'was_impossible': False}
751 1224 3.0 user: 751        item: 1224       r_ui = 3.00   est = 3.33   {'was_impossible': False}
979 1379 4.0 user: 979        item: 1379       r_ui = 4.00   est = 3.58   {'was_impossible': False}
666 1127 1.0 user: 666        item: 1127       r_ui = 1.00   est = 2.60   {'was_impossible': False}
95

In [11]:
from sklearn.metrics import mean_squared_error
mean_squared_error(list_actual, list_pred)  

1.6666468113124562

In [12]:
data_test = spark.read.text("C:\\Users\\kpgaj\\test.dat")
splitted_t = data_rdd.map(lambda row: row.value.split("\t"))

rrdd_t = splitted_t.map(lambda a: Row(UserID=int(a[0]),MovieID=int(a[1])))
data_frame_spark_T = spark.createDataFrame(rrdd_t).cache()
pd_test = data_frame_spark_T.toPandas()

In [13]:
pd_test_sample = pd_test[:2154]

In [14]:
f = open("format3.dat","w+")
f.write("RatingID,Rating\n")
count=0
for i,row in pd_test_sample.iterrows():
    count += 1
    
    f.write('{},{:.2f}\n'.format(count,(svdtuned.predict(row["MovieID"],row["UserID"]).est)))


f.close()